In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 54.7 MB/s eta 0:00:00


In [ ]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, LpMinimize, LpConstraintVar, PULP_CBC_CMD, CPLEX_CMD, GUROBI_CMD, COINMP_DLL, COIN_CMD
from pulp.apis import COINMP_DLL
import pulp
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

def fill_connected(connected, neighbours, r):
    connected.union(set(neighbours.copy()))
    for i in neighbours:
        connected.add(i)
        fill_connected(connected, r[i], r)
    return

def get_r_dist(r):
    r_dist = []
    for i in range(n):
        r_dist.append([])
        for j in range(2**len(r[i])):
            r_dist[i].append(set([r[i][k] for k in range(len(r[i])) if (j & (1 << k))]))
    return r_dist

# Define our parameters
n = 5 # excluding the seller
connected_to_seller = [0, 1] # nodes which are directly connected to the seller
r = [[2, 3, 4], [], [], [], []] # the neighbours of each node in the form of a list
epsilon = 0.5 # the values of v taken are from 0 to 1 with seaparation = epsilon 

assert len(r) == n

# find the nodes disconnected from the seller
connected = set()
fill_connected(connected, connected_to_seller, r)
disconnected = [i for i in range(n) if i not in connected]


v_dist = np.arange(0, 1 + epsilon, epsilon)
v_prob = [1/len(v_dist) for _ in range(n)]

# power set of each r_i
r_dist = get_r_dist(r)
# probability of each r_i
r_prob = [0 if len(r_dist[i]) == 0 else 1/len(r_dist[i]) for i in range(len(r_dist))]

theta = []
for i in range(n):
    theta.append([])
    for vi in v_dist:
        for ri in r_dist[i]:
            theta[i].append((vi, ri))

theta_minus = [] # denotes set of all theta_i other than agent i
for i in range(n):
    if n == 2:
        theta_minus.append([(el,) for el in theta[1-i]])
    elif i == 0:
        theta_minus.append(list(product(*theta[1:])))
    elif i == n-1:
        theta_minus.append(list(product(*theta[:i])))
    else:
        theta_minus.append(list(product(*theta[:i], *theta[i+1:])))

total_theta = list(product(*theta))
theta_minus_prob = [0 if len(theta_minus[i]) == 0 else 1/len(theta_minus[i]) for i in range(len(theta_minus))]

# Initialize our model
model = LpProblem(name="lp", sense=LpMaximize)

# Initialize our decision variables
print("Initializing decision variables")
alpha = []
for i in range(n):
    alpha.append({})
    for vi in v_dist:
        alpha[i][vi] = {}
        for ri in r_dist[i]:
            # convert ri to a string
            alpha[i][vi][str(ri)] = LpVariable(name=f"alpha_{i}_{vi}_{ri}", lowBound=0, upBound=1, cat='Continuous')

pay = []
for i in range(n):
    pay.append({})
    for vi in v_dist:
        pay[i][vi] = {}
        for ri in r_dist[i]:
            pay[i][vi][str(ri)] = LpVariable(name=f"pay_{i}_{vi}_{ri}", cat='Continuous')

g = []
for i in range(n):
    g.append({})
    for theta_i in theta[i]:
        g[i][str(theta_i)] = {}
        for theta_i_minus in theta_minus[i]:
            g[i][str(theta_i)][str(theta_i_minus)] = LpVariable(name=f"g_{i}_{theta_i}_{theta_i_minus}", cat='Binary')

# Add the objective function to the model
model.setObjective(lpSum([pay[i][vi][str(ri)]*v_prob[i]*r_prob[i] for i in range(n) for vi in v_dist for ri in r_dist[i]]))

# add constraints

# alpha definition constraint
print("Adding alpha definition constraints")
for i in range(n):
    for theta_i in theta[i]:
        vi = theta_i[0]
        ri = theta_i[1]
        model.addConstraint(alpha[i][vi][str(ri)] == lpSum([g[i][str(theta_i)][str(theta_i_minus)]*theta_minus_prob[i] for theta_i_minus in theta_minus[i]]))

# Total Constraint
print("Adding total constraint")
sum = 0
for theta in total_theta:
    sum = 0
    for i in range(n):
        theta_i = theta[i]
        theta_i_minus = None
        if n == 2:
            theta_i_minus = (theta[1-i],)
        elif i == 0:
            theta_i_minus = tuple(theta[1:])
        elif i == n-1:
            theta_i_minus = tuple(theta[:i])
        else:
            theta_i_minus = tuple(theta[:i]+theta[i+1:])
        sum += g[i][str(theta_i)][str(theta_i_minus)]
    model.addConstraint(sum == 1)

# Connected constraint
print("Adding connected constraint")
for i in disconnected:
    for vi in v_dist:
        for ri in r_dist[i]:
            model.addConstraint(alpha[i][vi][str(ri)] == 0)
            model.addConstraint(pay[i][vi][str(ri)] == 0)

# FFEP-b
print("Adding FFEP-b constraints")
for i in range(n):
    for vi in v_dist:
        for ri in r_dist[i]:
            if i == 0:
                model.addConstraint(pay[i][vi][str(ri)] == pay[i][0][str(ri)] + vi*alpha[i][vi][str(ri)] - epsilon*lpSum([alpha[i][j][str(ri)] for j in v_dist if j <= vi]))
            else:
                model.addConstraint(pay[i][vi][str(ri)] == pay[i][0][str(ri)] + vi*alpha[i][vi][str(ri)] - epsilon*lpSum([alpha[i][j][str(ri)] for j in v_dist if j < vi]))

# FFEP-a
print("Adding FFEP-a constraints")
for i in range(n):
    for vi in v_dist:
        for ri in r_dist[i]:
            for ri2 in r_dist[i]:
                # check if ri2 is a subset of ri
                if ri2.issubset(ri):
                    pseudo_connected = set()
                    new_r = r.copy()
                    new_r[i] = list(ri2)
                    fill_connected(pseudo_connected, connected_to_seller, new_r)
                    if i in pseudo_connected:
                        model.addConstraint(pay[i][0][str(ri2)] - pay[i][0][str(ri)] >= epsilon*lpSum([alpha[i][j][str(ri2)] - alpha[i][j][str(ri)] for j in v_dist if j < vi]))

# DBIC-1
print("Adding DBIC-1 constraints")
for i in range(n):
    for ri in r_dist[i]:
        for vi in v_dist:
            for vi2 in v_dist:
                if vi2 >= vi:
                    model.addConstraint(alpha[i][vi2][str(ri)] >= alpha[i][vi][str(ri)])

# IIR
print("Adding IIR constraints")
for i in range(n):
    for vi in v_dist:
        for ri in r_dist[i]:
            model.addConstraint(vi*alpha[i][vi][str(ri)] - pay[i][vi][str(ri)] >= 0)


Initializing decision variables
Adding alpha definition constraints
Adding total constraint
Adding connected constraint
Adding FFEP-b constraints
Adding FFEP-a constraints
Adding DBIC-1 constraints
Adding IIR constraints


In [ ]:

# Solve our problem
print("Solving...")
solver = PULP_CBC_CMD(msg=True)
status = model.solve(solver)
max_value = model.objective.value()

# Print our decision variable values
for i in range(n):
    for vi in v_dist:
        for ri in r_dist[i]:
            print(f"alpha_{i}_{vi}_{ri} = {alpha[i][vi][str(ri)].value()}")
            print(f"pay_{i}_{vi}_{ri} = {pay[i][vi][str(ri)].value()}")
print(f"Optimal value: {round(model.objective.value(), 4)}")
print(f"Status: {model.status}, {LpStatus[model.status]}")

Solving...
alpha_0_0.0_set() = 0.0
pay_0_0.0_set() = 0.0
alpha_0_0.0_{2} = 0.0
pay_0_0.0_{2} = 0.0
alpha_0_0.0_{3} = 0.0
pay_0_0.0_{3} = 0.0
alpha_0_0.0_{2, 3} = 0.0
pay_0_0.0_{2, 3} = 0.0
alpha_0_0.0_{4} = 0.0
pay_0_0.0_{4} = 0.0
alpha_0_0.0_{2, 4} = 0.0
pay_0_0.0_{2, 4} = 0.0
alpha_0_0.0_{3, 4} = 0.0
pay_0_0.0_{3, 4} = 0.0
alpha_0_0.0_{2, 3, 4} = 0.0
pay_0_0.0_{2, 3, 4} = 0.0
alpha_0_0.5_set() = 0.012345679
pay_0_0.5_set() = 0.0
alpha_0_0.5_{2} = 0.012345679
pay_0_0.5_{2} = 0.0
alpha_0_0.5_{3} = 0.012345679
pay_0_0.5_{3} = 0.0
alpha_0_0.5_{2, 3} = 0.012345679
pay_0_0.5_{2, 3} = 0.0
alpha_0_0.5_{4} = 0.012345679
pay_0_0.5_{4} = 0.0
alpha_0_0.5_{2, 4} = 0.012345679
pay_0_0.5_{2, 4} = 0.0
alpha_0_0.5_{3, 4} = 0.012345679
pay_0_0.5_{3, 4} = 0.0
alpha_0_0.5_{2, 3, 4} = 0.012345679
pay_0_0.5_{2, 3, 4} = 0.0
alpha_0_1.0_set() = 0.19753086
pay_0_1.0_set() = 0.092592593
alpha_0_1.0_{2} = 0.19753086
pay_0_1.0_{2} = 0.092592593
alpha_0_1.0_{3} = 0.19753086
pay_0_1.0_{3} = 0.092592593
alpha_0_1.